In [1]:
import pybaseball
import pandas as pd

pd.set_option('display.max_rows', None)

# Data

In [2]:
d = pybaseball.statcast()
d.shape

start_dt 2025-05-03
end_dt 2025-05-04
This is a large query, it may take a moment to complete


  0%|                                                                               | 0/2 [00:00<?, ?it/s]/Users/andrewenfield/work/github/mlb_ml/.venv/lib/python3.11/site-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
100%|███████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.56s/it]
/Users/andrewenfield/work/github/mlb_ml/.venv/lib/python3.11/site-packages/pybaseball/statcast.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_da

(4042, 113)

In [3]:
d[:10]

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle
2086,CU,2025-05-03,83.0,-2.27,5.98,"Ross, Joe",677950,605452,strikeout,swinging_strike_blocked,...,1,3,<NA>,<NA>,<NA>,<NA>,3.64,-0.14,0.14,<NA>
2139,SI,2025-05-03,94.8,-2.37,5.93,"Ross, Joe",677950,605452,NaN,ball,...,1,3,<NA>,<NA>,<NA>,<NA>,1.53,1.23,-1.23,<NA>
2174,FF,2025-05-03,92.0,-2.38,5.92,"Ross, Joe",677950,605452,NaN,foul_tip,...,1,3,<NA>,<NA>,<NA>,<NA>,1.67,0.23,-0.23,<NA>
2244,FF,2025-05-03,93.4,-2.51,5.89,"Ross, Joe",677950,605452,NaN,ball,...,1,3,<NA>,<NA>,<NA>,<NA>,1.6,0.15,-0.15,<NA>
2347,CH,2025-05-03,89.4,-2.38,5.85,"Ross, Joe",677950,605452,NaN,called_strike,...,1,3,<NA>,<NA>,<NA>,<NA>,2.25,1.23,-1.23,<NA>
2376,FF,2025-05-03,95.3,-2.2,5.94,"Ross, Joe",666971,605452,double,hit_into_play,...,1,3,<NA>,<NA>,<NA>,<NA>,1.84,0.83,0.83,<NA>
2455,SL,2025-05-03,86.8,-2.36,5.87,"Ross, Joe",666971,605452,NaN,foul,...,1,3,<NA>,<NA>,<NA>,<NA>,2.83,-0.32,-0.32,<NA>
2507,FF,2025-05-03,93.7,-2.21,5.99,"Ross, Joe",666971,605452,NaN,foul,...,1,3,<NA>,<NA>,<NA>,<NA>,1.59,0.34,0.34,<NA>
2642,FF,2025-05-03,93.4,-2.26,5.87,"Ross, Joe",666971,605452,NaN,called_strike,...,1,3,<NA>,<NA>,<NA>,<NA>,1.73,0.08,0.08,<NA>
2685,FF,2025-05-03,94.5,-2.46,5.83,"Ross, Joe",666971,605452,NaN,foul,...,1,3,<NA>,<NA>,<NA>,<NA>,1.71,0.09,0.09,<NA>


Field documentation is at https://baseballsavant.mlb.com/csv-docs.

Rough quickie initial notes on relevant fields:

- pitch_type for ground truth (how is 'pitch_name' same/diff?)
- potentially and/or likely useful for prediction:
    - release_speed
    - pitcher - MLB player ID, for per-pitcher learning
    - pfx_x and pfx_z - horiz/vert movement in feet from catcher's perspective
    - plate_x and plate_z - horiz/vert position of ball when it crosses home place; how different in info from pfx?
    - vx0, vy0, vz0 - velocity of pitch in ft/sec, in each dimension at y=50 ft; how diff in info from release_speed?
    - ax, ay, az - acceleration of pitch in ft/sec, in each dimension at y=50 ft
    - effective_speed - 'derived speed based on the extension of the pitcher's release'
    - release_spin - spin rate of pitch
    - spin_axis - spin axis in the 2D X-Z plane in degrees from 0-360, where 180 is a 'pure backspin fastball' and 0 is a 'pure topsping - 12-6 - curveball'
    - release_extension - 'release extension' in feet
    - if_fielding_alignment/of_fielding_alignment - infield/outfield fielding alignment at time of pitch
- maybe useful, for ex w/ some addtl preprocessing or other work, or given more thought:
    - release_pos_x and release_pos_z
    - stand - side of plate batter is standing
    - p_throws - hand pitcher throws with
    - balls - pre-pitch number of balls in count
    - strikes - pre-pitch number of strikes in count
    - game_year - year game took place (change over time?)
    - outs_when_up - pre-pitch number of outs
    - inning - pre-pitch inning number
    - on_1b, on_2b, on_3b - MLB player IDs of runners on base
    - release_pos_y - 'release position' of pitch in feet from catcher's perspective
    - pitch_number - total pitch number of the plate appearance
    - bat_score and fld_score - pre-pitch batting/fielding team scores

Seems like there are a relatively small number of fields that most directly relate to the kind of pitch - things like release speed, movement, and spin rate/axis, as well as the actual pitcher given that folks typically throw different pitches and with different distributions over the pitches thrown - and also a potentially somewhat larger set of fields that could be correlated with changes in pitches thrown/give hints at what a pitch could be/likelihood of a particular pitch being thrown in different cases - like the game situation (runners on base, balls/strikes, etc.).   

In [4]:
d.iloc[0]

pitch_type                                                                         CU
game_date                                                         2025-05-03 00:00:00
release_speed                                                                    83.0
release_pos_x                                                                   -2.27
release_pos_z                                                                    5.98
player_name                                                                 Ross, Joe
batter                                                                         677950
pitcher                                                                        605452
events                                                                      strikeout
description                                                   swinging_strike_blocked
spin_dir                                                                         <NA>
spin_rate_deprecated                                  

In [5]:
d['pitch_type'].value_counts(dropna=False)

pitch_type
FF    1223
SI     646
SL     562
CH     416
ST     339
FC     259
FS     258
CU     229
KC      91
SV      19
Name: count, dtype: int64

# Preprocessing